### Метрики и разбиение данных

In [1]:
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

cancer = datasets.load_breast_cancer()
logistics_regression = LogisticRegression()
model = logistics_regression.fit(cancer.data, cancer.target)

print('Accuracy: {:.2f}'.format(model.score(cancer.data, cancer.target)))

Accuracy: 0.95


/Users/linuxoid/.virtualenvs/dl4cv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [2]:
predictions = model.predict(cancer.data)

print('Accuracy: {:.2f}'.format(metrics.accuracy_score(cancer.target, predictions)))
print('ROC AUC: {:.2f}'.format(metrics.roc_auc_score(cancer.target, predictions)))
print('F1: {:.2f}'.format(metrics.f1_score(cancer.target, predictions)))

Accuracy: 0.95
ROC AUC: 0.94
F1: 0.96


In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size = 0.2, random_state = 12)

print('Train accuracy: {:.2f}'.format(model.score(x_train, y_train)))
print('Test accuracy: {:.2f}'.format(model.score(x_test, y_test)))

Train accuracy: 0.95
Test accuracy: 0.94


Мы будем делать это с помощью какого то случайного сида random_state 12, для того, чтобы у нас зафиксировало качество раз навсегда

на тесте у нас качество чуть хуже. Это достаточно логичная ситуация, мы обучались на одних данных а смотрим качество немного других. Если у нас отличия небольшое как в данном случае, это не страшно. Если у нас был например 0,6, то скорее всего можно считать, что модель у нас переобучилась под тренировочную выборку.

In [4]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet

boston = datasets.load_boston()

lasso = Lasso() # L1 regul
ridge = Ridge() # L2 
elastic = ElasticNet() # L1 and L2

for model in (lasso, ridge, elastic):
    x_train, x_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target,
    test_size = 0.2
    )
    model.fit(x_train, y_train)
    
    predictions = model.predict(x_test)
    print(model.__class__)
    print('MSE: {:.2f}\n'.format(metrics.mean_squared_error(y_test, predictions)))

<class 'sklearn.linear_model._coordinate_descent.Lasso'>
MSE: 0.11

<class 'sklearn.linear_model._ridge.Ridge'>
MSE: 0.06

<class 'sklearn.linear_model._coordinate_descent.ElasticNet'>
MSE: 0.09



In [5]:
print('R2: {:.2f}'.format(model.score(x_test, y_test)))
print('R2: {:.2f}'.format(metrics.r2_score(y_test, predictions)))

R2: 0.59
R2: 0.59


### Кросс-валидация

использовать все данные для обучения, делается это так, чтобы знать что наша модель не переобучается и работает хорошо

kFold, класс который позволяет вам делать как раз кросс-валидацию

In [6]:
from sklearn.model_selection import KFold, cross_val_score

iris = datasets.load_iris()
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

Решает задачу классификации. У нас четыре признака у цветка: длина лепестка, ширина лепестка, длина чашелистника и его ширина.

У нас три класса:
 
- Iris-Setosa
- Iris-Versicolour
- Iris-Vi
                
мы пытаемся угадать к какому из классов относится цветок по его характеристикам.

In [7]:
print(iris.DESCR[:475])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Vi


In [8]:
logistics_regression = LogisticRegression()

# выборка разбивается на блоки, у нас их 5
# 4 блока для обучения, 1 для оценки качества
# потом берем след четыре блока
# и какой-то новый из этих пяти блоков
# выступает в качестве революционного блока и тд
# то есть в цикле итерируемся по разным разбиениям

cv = KFold(n_splits = 5) # StratifiedFold

for split_idx, (train_idx, test_idx) in enumerate(cv.split(iris.data)):
    x_train, x_test = iris.data[train_idx], iris.data[test_idx]
    y_train, y_test = iris.target[train_idx], iris.target[test_idx]

    logistics_regression.fit(x_train, y_train)
    score = logistics_regression.score(x_test, y_test)
    print('Split {} Score: {:.2f}'.format(split_idx, score))

/Users/linuxoid/.virtualenvs/dl4cv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/linuxoid/.virtualenvs/dl4cv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

Split 0 Score: 1.00
Split 1 Score: 1.00
Split 2 Score: 0.87
Split 3 Score: 0.93
Split 4 Score: 0.83


In [9]:
cv_score = cross_val_score(
    logistics_regression, iris.data, iris.target,
    scoring = 'accuracy', cv = cv)

print('Cross val score: {}'.format(cv_score))
print('Mean cross val score: {:.2f}'.format(cv_score.mean()))

Cross val score: [1.         1.         0.86666667 0.93333333 0.83333333]
Mean cross val score: 0.93


/Users/linuxoid/.virtualenvs/dl4cv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/linuxoid/.virtualenvs/dl4cv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

Test

In [11]:
TP = 15
FP = 5

TN = 920
FN = 60

accuracy = (TP + TN) / (TP + FP + TN + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = (2 * precision * recall) / (precision + recall)

print('accuracy: {}'.format(accuracy))
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('f1: {}'.format(f1))

accuracy: 0.935
precision: 0.75
recall: 0.2
f1: 0.31578947368421056


In [12]:
1/3

0.3333333333333333

In [14]:
precision / recall

3.75